In [2]:
import pandas as pd
import sqlite3

In [7]:
db_path = "../data/traffic_data.db"
conn=sqlite3.connect(db_path)
df=pd.read_sql_query('select * from traffic_data',conn)

print("Data Shape:", df.shape)
print(df.head())

Data Shape: (1146, 9)
   id location_name      lat      lon                   timestamp  \
0   1         Dadar  19.0176  72.8562  2025-07-23T23:25:24.659867   
1   2       Andheri  19.1197  72.8468  2025-07-23T23:25:25.672796   
2   3         Powai  19.1197  72.9051  2025-07-23T23:25:26.528793   
3   4        Colaba  18.9067  72.8147  2025-07-23T23:25:27.154680   
4   5        Panvel  18.9894  73.1175  2025-07-23T23:25:27.893369   

   current_speed  free_flow_speed  confidence road_class  
0           27.0             27.0         1.0    Unknown  
1           22.0             22.0         1.0    Unknown  
2           31.0             31.0         1.0    Unknown  
3           23.0             23.0         1.0    Unknown  
4           21.0             21.0         1.0    Unknown  


In [5]:
df

,id,location_name,lat,lon,timestamp,current_speed,free_flow_speed,confidence,road_class
0,1,Dadar,19.0176,72.8562,2025-07-23T23:25:24.659867,27.0,27.0,1.000000,Unknown
1,2,Andheri,19.1197,72.8468,2025-07-23T23:25:25.672796,22.0,22.0,1.000000,Unknown
2,3,Powai,19.1197,72.9051,2025-07-23T23:25:26.528793,31.0,31.0,1.000000,Unknown
3,4,Colaba,18.9067,72.8147,2025-07-23T23:25:27.154680,23.0,23.0,1.000000,Unknown
4,5,Panvel,18.9894,73.1175,2025-07-23T23:25:27.893369,21.0,21.0,1.000000,Unknown
...,...,...,...,...,...,...,...,...,...
1141,1142,Panvel,18.9894,73.1175,2025-07-26T16:33:18.137767,21.0,21.0,1.000000,Unknown
1142,1143,Airoli,19.1551,72.9960,2025-07-26T16:33:18.865597,13.0,15.0,0.871303,Unknown
1143,1144,Vashi,19.0760,72.9986,2025-07-26T16:33:19.355936,19.0,24.0,0.997606,Unknown
1144,1145,Nerul,19.0334,73.0186,2025-07-26T16:33:19.870760,19.0,19.0,1.000000,Unknown


In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

# ➤ Add only required derived columns
df['is_weekend'] = df['timestamp'].dt.weekday.isin([5,6]).astype(int)
df['is_rush_hour'] = df['timestamp'].dt.hour.isin([7,8,9,17,18,19]).astype(int)
df['congestion_level'] = df['free_flow_speed'] - df['current_speed']
df['traffic_ratio'] = df['current_speed'] / df['free_flow_speed']

# Replace infinities & NaNs in ratio
df['traffic_ratio'] = df['traffic_ratio'].replace([float('inf'), -float('inf')], None)

# ➤ Save as enriched table
df.to_sql("traffic_data_enriched", conn, if_exists="replace", index=False)

conn.close()
print("✅ Enriched data saved to table 'traffic_data_enriched'")

✅ Enriched data saved to table 'traffic_data_enriched'
